<a href="https://colab.research.google.com/github/Priyansh-Kedia/AmazonChallenge/blob/master/AmazonChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Add all imports here

from nltk.corpus import stopwords

In [2]:
# Download dataset

!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip

--2021-07-30 04:59:55--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.124.106
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.124.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  17.4MB/s    in 62s     

2021-07-30 05:00:58 (16.3 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]



In [3]:
# Unzip the dataset

!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


In [2]:
# Import the dataset in pandas

import pandas as pd
import csv
df = pd.read_csv("dataset/train.csv", low_memory=True, escapechar="\\", quoting=csv.QUOTE_NONE)

In [5]:
# Add the column variables here

TITLE = "TITLE"
PRODUCT_ID = "PRODUCT_ID"
DESCRIPTION = "DESCRIPTION"
BULLET_POINTS = "BULLET_POINTS"
BRAND = "BRAND"
BROWSE_NODE_ID = "BROWSE_NODE_ID"

In [6]:
# Load the doc
def load_doc(df):
  titles = df[TITLE].tolist()
  print(titles)

load_doc(df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

